In [1]:
!pip install seisbench

In [ ]:
import os
import obspy
from obspy import read
import seisbench.models as sbm

# Load the model
model = sbm.DeepDenoiser.from_pretrained("original")

# Define the source and target directories
source_directory = '/Volumes/Marco/concurrent_denoiser'
target_directory = '/Volumes/Marco/concurrent_denoiser/denoised'

# Ensure the target directory exists
os.makedirs(target_directory, exist_ok=True)

# Get all miniseed files
mseed_files = [f for f in os.listdir(source_directory) if f.endswith('.mseed')]

# Sort files by their names
mseed_files.sort()

# Loop through each sorted file
for filename in mseed_files:
    file_path = os.path.join(source_directory, filename)
    
    # Read the stream
    stream = read(file_path)

    # Apply the model to denoise
    denoised = model.annotate(stream)

    # Edit Traces
    for trace in denoised:
        if 'DeepDenoiser_' in trace.stats.channel:
            trace.stats.channel = trace.stats.channel.replace('DeepDenoiser_', '')
    
    target_file_path = os.path.join(target_directory, filename)
    
    # Create a new Stream object to contain all denoised traces
    denoised_stream = obspy.Stream(traces=denoised)

    # Write denoised signals to the target directory
    denoised_stream.write(target_file_path, format='MSEED')

    print(f'Processed and saved {filename}')



In [ ]:
import os
import obspy
from obspy import read
import seisbench.models as sbm

In [ ]:
import os
import obspy
from obspy import read
import seisbench.models as sbm
from multiprocessing import Pool

# Function to process each file
def process_file(filename):
    file_path = os.path.join(source_directory, filename)
    stream = read(file_path)
    denoised = model.annotate(stream)
    for trace in denoised:
        if 'DeepDenoiser_' in trace.stats.channel:
            trace.stats.channel = trace.stats.channel.replace('DeepDenoiser_', '')
    target_file_path = os.path.join(target_directory, filename)
    denoised_stream = obspy.Stream(traces=denoised)
    denoised_stream.write(target_file_path, format='MSEED')
    print(f'Processed and saved {filename}')

if __name__ == '__main__':
    # Load the model
    model = sbm.DeepDenoiser.from_pretrained("original")

    # Define the source and target directories
    source_directory = '/Volumes/Marco/concurrent_denoiser'
    target_directory = '/Volumes/Marco/concurrent_denoiser/denoised'

    # Ensure the target directory exists
    os.makedirs(target_directory, exist_ok=True)

    # Get all miniseed files
    mseed_files = [f for f in os.listdir(source_directory) if f.endswith('.mseed')]

    # Sort files by their names
    mseed_files.sort()

    # Define the number of processes (adjust as needed)
    num_processes = os.cpu_count() - 1

    # Create a Pool of worker processes
    with Pool(processes=num_processes) as pool:
        # Map the process function to the list of files
        pool.map(process_file, mseed_files)
